In [21]:
# Modules to support development
import os
import re
import collections
import itertools
import functools
import logging
import pprint
import numpy as np
import heapq
import copy
from tqdm import tqdm
import math
import nographs as nog

In [22]:
class Blueprint():
    def __init__(self):
        self.id = None
        self.costs = {}

    def parse_blueprint(self, blueprint):
        mm = re.match("Blueprint (\d+): (.*)", blueprint)
        if not mm:
            raise ValueError

        self.id = int( mm.group(1) )
        costs = mm.group(2)
        
        for cost_ll in costs.split("."):
            if cost_ll.strip() == "":
                continue

            mm = re.match("Each (\w+) robot costs (\d+) (\w+)(?:\s+and (\d+) (\w+))?", cost_ll.strip())
            if not mm:
                raise ValueError(cost_ll)
            what = mm.group(1)
            cost = int(mm.group(2))
            item = mm.group(3)

            self.costs.setdefault(what, {})[item] = cost
            if mm.group(4) is not None:
                cost = int(mm.group(4))
                item = mm.group(5)
                self.costs.setdefault(what, {})[item] = cost

def test_parse_blueprint(): 
    b = Blueprint()
    b.parse_blueprint("Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 2 ore and 11 clay. Each geode robot costs 4 ore and 8 obsidian.")
    assert b.id == 1
    print(b.costs)

def read_input(puzzle_input):
    with open(puzzle_input) as ff:
        dd = ff.readlines()

    blueprints = []
    for ll in dd:
        bb = Blueprint()
        bb.parse_blueprint(ll)
        blueprints.append(bb)

    return blueprints

def test_read_input():
    blueprints = read_input(os.path.join(os.path.join("..", "dat", "day19_test.txt")))
    for bb in blueprints:
        print(bb.id, bb.costs)

test_read_input()


1 {'ore': {'ore': 4}, 'clay': {'ore': 2}, 'obsidian': {'ore': 3, 'clay': 14}, 'geode': {'ore': 2, 'obsidian': 7}}
2 {'ore': {'ore': 2}, 'clay': {'ore': 3}, 'obsidian': {'ore': 3, 'clay': 8}, 'geode': {'ore': 3, 'obsidian': 12}}


In [54]:
State = collections.namedtuple("State", "ore clay obsidian geode ore_r clay_r obsidian_r geode_r")

def next_state(state, blueprint):
    can_build_all = True
    for build in ("ore", "clay", "obsidian"):
        remaining_ore = state.ore_r - blueprint.costs.get(build, {}).get("ore", 0)
        remaining_clay = state.clay_r - blueprint.costs.get(build, {}).get("clay", 0)
        remaining_obsidian = state.obsidian_r - blueprint.costs.get(build, {}).get("obsidian", 0)
        can_build_all = remaining_ore >= 0 and remaining_clay >= 0 and remaining_obsidian >= 0
        if can_build_all == False:
            break

    if can_build_all:
        build_list = (None, "geode")
    else:
        build_list = (None, "ore", "clay", "obsidian", "geode")

    for build in build_list:
        remaining_ore = state.ore - blueprint.costs.get(build, {}).get("ore", 0)
        remaining_clay = state.clay - blueprint.costs.get(build, {}).get("clay", 0)
        remaining_obsidian = state.obsidian - blueprint.costs.get(build, {}).get("obsidian", 0)
        can_build = remaining_ore >= 0 and remaining_clay >= 0 and remaining_obsidian >= 0

        if build is None:   
            yield build, State(
                #state.depth + 1,
                remaining_ore + state.ore_r,
                remaining_clay + state.clay_r,
                remaining_obsidian + state.obsidian_r,
                state.geode + state.geode_r,
                state.ore_r,
                state.clay_r,
                state.obsidian_r,
                state.geode_r,
            )
        elif can_build == False:
            continue
        elif build == "ore":
            yield build, State(
                #state.depth + 1,
                remaining_ore + state.ore_r,
                remaining_clay + state.clay_r,
                remaining_obsidian + state.obsidian_r,
                state.geode + state.geode_r,
                state.ore_r + 1,
                state.clay_r,
                state.obsidian_r,
                state.geode_r,
            )
        elif build == "clay":
            yield build, State(
                #state.depth + 1,
                remaining_ore + state.ore_r,
                remaining_clay + state.clay_r,
                remaining_obsidian + state.obsidian_r,
                state.geode + state.geode_r,
                state.ore_r,
                state.clay_r + 1,
                state.obsidian_r,
                state.geode_r,
            )
        elif build == "obsidian":
            yield build, State(
                #state.depth + 1,
                remaining_ore + state.ore_r,
                remaining_clay + state.clay_r,
                remaining_obsidian + state.obsidian_r,
                state.geode + state.geode_r,
                state.ore_r,
                state.clay_r,
                state.obsidian_r + 1,
                state.geode_r,
            )
        elif build == "geode":
            yield build, State(
                #state.depth + 1,
                remaining_ore + state.ore_r,
                remaining_clay + state.clay_r,
                remaining_obsidian + state.obsidian_r,
                state.geode + state.geode_r,
                state.ore_r,
                state.clay_r,
                state.obsidian_r,
                state.geode_r + 1,
            )            

def test_next_state():
    blueprints = read_input(os.path.join(os.path.join("..", "dat", "day19_test.txt")))
    state = State(0, 0, 0, 0, 0, 1, 0, 0, 0)
    states = list(next_state(state, blueprints[0]))
    assert(len(states) == 1)
    assert states[0] == State(1, 1, 0, 0, 0, 1, 0, 0, 0)
    
    state = State(1, 1, 0, 0, 0, 1, 0, 0, 0)
    states = list(next_state(state, blueprints[0]))
    assert(len(states) == 1)
    assert states[0] == State(2, 2, 0, 0, 0, 1, 0, 0, 0)

    state = State(2, 2, 0, 0, 0, 1, 0, 0, 0)
    states = list(next_state(state, blueprints[0]))
    assert(len(states) == 2)
    assert states[0] == State(3, 3, 0, 0, 0, 1, 0, 0, 0)
    assert states[1] == State(3, 1, 0, 0, 0, 1, 1, 0, 0)

    state = State(3, 4, 0, 0, 0, 1, 1, 0, 0)
    states = list(next_state(state, blueprints[0]))
    assert(len(states) == 3)
    assert states[0] == State(4, 5, 1, 0, 0, 1, 1, 0, 0) # Build None
    assert states[1] == State(4, 1, 1, 0, 0, 2, 1, 0, 0) # Build ore
    assert states[2] == State(4, 3, 1, 0, 0, 1, 2, 0, 0) # Build clay

    state = State(10, 4, 15, 0, 0, 1, 3, 0, 0)
    states = list(next_state(state, blueprints[0]))
    assert(len(states) == 4)
    assert states[0] == State(11, 5, 18, 0, 0, 1, 3, 0, 0) # Build None
    assert states[1] == State(11, 1, 18, 0, 0, 2, 3, 0, 0) # Build ore
    assert states[2] == State(11, 3, 18, 0, 0, 1, 4, 0, 0) # Build clay
    assert states[3] == State(11, 2, 4, 0, 0, 1, 3, 1, 0) # Build obsidian

    state = State(17, 3, 13, 8, 0, 1, 4, 2, 0)
    states = list(next_state(state, blueprints[0]))
    assert(len(states) == 3)
    assert states[0] == State(18, 4, 17, 10, 0, 1, 4, 2, 0) # Build None
    assert states[1] == State(18, 2, 17, 10, 0, 1, 5, 2, 0) # Build clay
    assert states[2] == State(18, 2, 17, 3,  0, 1, 4, 2, 1) # Build geode

    state = State(24,   6, 41, 8, 9,   1, 4, 2, 2)
    states = list(next_state(state, blueprints[0]))
    assert(len(states) == 5)
    assert states[0] == State(25,   7, 45, 10, 11,   1, 4, 2, 2) # Build None
    assert states[1] == State(25,   3, 45, 10, 11,   2, 4, 2, 2) # Build ore
    assert states[2] == State(25,   5, 45, 10, 11,   1, 5, 2, 2) # Build clay
    assert states[3] == State(25,   4, 31, 10, 11,   1, 4, 3, 2) # Build obsidian
    assert states[4] == State(25,   5, 45, 3,  11,   1, 4, 2, 3) # Build geode

#test_next_state()

In [77]:
def score(state, rounds_left, blueprint):
    # The current score is number of geodes, plus assuming we will generate a geode for each robot
    current_score = state.geode + state.geode_r * rounds_left

    #can_build_geode = (blueprint.costs['geode']['ore'] >= state.ore) or (blueprint.costs['geode']['obsidian'] >= state.obsidian)
    #best_score = 0
    #if state.geode_r > 0 or can_build_geode:
        # Best score is how many geodes we get if we produce a new geode robot each remaining round:
        # sum (n-1)+(n-2)+(n-3)
    n = rounds_left - 1
    best_score = current_score + ((n*(n+1)) // 2)

    return current_score, best_score

In [85]:
def analyze(blueprint, max_depth):
    global max_score, prune_cnt
    max_score = 0
    prune_cnt = 0

    def next_edges(state, t):
        global max_score, prune_cnt
        minutes_left = max_depth - t.depth

        if minutes_left == 0:
            return

        _, best_score = score(state, minutes_left, blueprint)
        if best_score < max_score:
            prune_cnt += 1
            return 

        next_states = list(next_state(state, blueprint))
        for build, child in next_states:
            child_score, _ = score(child, minutes_left-1, blueprint)
            if child_score > max_score:
                max_score = max(max_score, child_score)
                print("New Max Score", max_score, child, t.depth)
            yield child, 0 if build == "geode" else minutes_left

    traversal = nog.TraversalShortestPaths(next_edges)
    max_geodes = State(0, 0, 0, 0, 1, 0, 0, 0)

    state = State(0, 0, 0, 0, 1, 0, 0, 0)

    last_depth = 0
    for vertex in traversal.start_from(state):       
        #print(vertex, sum([ valves[xx][0] for xx in vertex[2] ]), traversal.depth)
        if vertex.geode > max_geodes.geode:
            max_geodes = vertex
            v_score, v_best_score = score(vertex, max_depth-traversal.depth, blueprint)
            print("MAX", max_geodes, v_score, v_best_score, traversal.depth, prune_cnt)
        #if last_depth != traversal.depth:
        #    print("DEPTH", traversal.depth, prune_cnt, max_score)
        #    last_depth = traversal.depth
    #for ii, path in enumerate(traversal.paths[max_flow]): 
    #    print(ii+1, path)

    return max_geodes.geode

def part2(puzzle_input):
    blueprints = read_input(puzzle_input)
    anss = [ 1 ] * 3
    for ii, blueprint in enumerate(blueprints[0:3]):
        print("="*80)
        ans = analyze(blueprint, 32)
        anss[ii] = ans

    print(anss)
    return anss[0] * anss[1] * anss[2]

ans = part2(os.path.join(os.path.join("..", "dat", "day19_test.txt")))
print(ans)
assert ans == 3472

ans = part2(os.path.join(os.path.join("..", "dat", "day19.txt")))
print(ans)

New Max Score 14 State(ore=2, clay=20, obsidian=2, geode=0, ore_r=1, clay_r=4, obsidian_r=2, geode_r=1) 17
MAX State(ore=1, clay=22, obsidian=4, geode=1, ore_r=1, clay_r=5, obsidian_r=2, geode_r=1) 14 92 19 0
MAX State(ore=1, clay=14, obsidian=6, geode=2, ore_r=1, clay_r=4, obsidian_r=3, geode_r=1) 14 80 20 0
New Max Score 25 State(ore=3, clay=29, obsidian=2, geode=3, ore_r=1, clay_r=4, obsidian_r=2, geode_r=2) 20
MAX State(ore=3, clay=29, obsidian=2, geode=3, ore_r=1, clay_r=4, obsidian_r=2, geode_r=2) 25 80 21 0
MAX State(ore=2, clay=35, obsidian=4, geode=5, ore_r=1, clay_r=5, obsidian_r=2, geode_r=2) 25 70 22 0
New Max Score 32 State(ore=2, clay=21, obsidian=6, geode=5, ore_r=2, clay_r=7, obsidian_r=5, geode_r=3) 22
New Max Score 39 State(ore=2, clay=28, obsidian=5, geode=7, ore_r=2, clay_r=7, obsidian_r=5, geode_r=4) 23
MAX State(ore=2, clay=28, obsidian=5, geode=7, ore_r=2, clay_r=7, obsidian_r=5, geode_r=4) 39 67 24 0
MAX State(ore=1, clay=27, obsidian=3, geode=9, ore_r=1, clay_r

OLD CODE
========

In [20]:
def analyze(blueprint, max_depth):
    Q = collections.deque()
    
    max_geode = 0
    max_score = [ 0 ] * 33
    prune_cnt = 0

    root = State(0, 0, 0, 0, 0, 1, 0, 0, 0)
    Q.append(root)

    earliest_geode = None

    while len(Q) > 0:
        v = Q.pop()  # BFS

        _, v_best_score = score(v, max_depth - v.depth, blueprint)
        
        if v.depth >= max_depth:
            continue
            
        if v_best_score < max_score[v.depth]:
            prune_cnt += 1
            continue

        #if (earliest_geode is None and v.geode) or (v.geode > 0 and earliest_geode > v.depth):
        #    earliest_geode = v.depth
        #    print("New earliest", max_geode, current_score, best_score, v)
        #elif v.geode == 0 and earliest_geode is not None and v.depth > earliest_geode:
        #    prune_cnt += 1
        #    continue

        next_states = list(next_state(v, blueprint))
        for child in next_states:
            current_score, best_score = score(child, max_depth - child.depth, blueprint)
            if max_geode < child.geode:
                print("New max geode", max_geode, current_score, best_score, child)
                max_geode = child.geode
            if max_score[child.depth] < current_score:
                print("New max score", max_geode, current_score, best_score, child)
                max_score[child.depth] = current_score
            Q.append(child)

    print("Pruned", prune_cnt)
    return max_geode

def part2(puzzle_input):
    blueprints = read_input(puzzle_input)
    total_quality = [1,1,1]
    for ii, blueprint in enumerate(blueprints[0:3]):
        print("-"*40)
        max_geodes = analyze(blueprint, 32)
        total_quality[ii] = max_geodes
            
    print(total_quality)
    return total_quality[0] * total_quality[1] * total_quality[2]

def test_part2():
    ans = part2(os.path.join(os.path.join("..", "dat", "day19_test.txt")))
    #print(ans)
    #assert ans == (56*62)

test_part2()

#print("X"*40)
#ans = part2(os.path.join(os.path.join("..", "dat", "day19.txt")))
#print(ans) # NOT 5130 (too low)

----------------------------------------
New max score 0 1 1 State(depth=31, ore=1, clay=167, obsidian=2, geode=0, ore_r=1, clay_r=11, obsidian_r=2, geode_r=1)
New max geode 0 1 1 State(depth=32, ore=2, clay=178, obsidian=4, geode=1, ore_r=1, clay_r=11, obsidian_r=2, geode_r=1)
New max score 1 1 1 State(depth=32, ore=2, clay=178, obsidian=4, geode=1, ore_r=1, clay_r=11, obsidian_r=2, geode_r=1)
New max score 1 2 3 State(depth=30, ore=2, clay=152, obsidian=2, geode=0, ore_r=1, clay_r=10, obsidian_r=2, geode_r=1)
New max score 1 2 2 State(depth=31, ore=3, clay=162, obsidian=4, geode=1, ore_r=1, clay_r=10, obsidian_r=2, geode_r=1)
New max geode 1 2 2 State(depth=32, ore=2, clay=173, obsidian=6, geode=2, ore_r=1, clay_r=11, obsidian_r=2, geode_r=1)
New max score 2 2 2 State(depth=32, ore=2, clay=173, obsidian=6, geode=2, ore_r=1, clay_r=11, obsidian_r=2, geode_r=1)
New max score 2 3 6 State(depth=29, ore=1, clay=139, obsidian=2, geode=0, ore_r=1, clay_r=10, obsidian_r=2, geode_r=1)
New max

KeyboardInterrupt: 

In [24]:
class State():
    def __init__(self, blueprint, prior_state=None):
        self.depth = 0
        self.inventory = {}
        self.robots = { "ore": 1 }
        #self.path = []
        self.prior_state = prior_state
        self.discovered = False
        self.blueprint = blueprint
        self.score = 0
        if prior_state is not None:
            self.depth = prior_state.depth + 1
            self.inventory.update(prior_state.inventory)
            self.robots.update(prior_state.robots)
            #self.path.extend(prior_state.path)

    def __lt__(self, other):
        # backwards for heapq descending order
        for item in ("geode", "obsidian", "clay", "ore"):
            if self.robots.get(item, 0) != other.robots.get(item, 0):
                return self.robots.get(item, 0) > other.robots.get(item, 0)

        return self.robots.get("geode", 0) > other.robots.get("geode", 0)
        
    def best_score(self, max_depth):
        n = max_depth - self.depth - 1 # rounds left
        geodes_per_robot = (n*(n+1))//2 # assume we build a geode robot per turn
        return (
            self.inventory.get("geode", 0) +  # current geodes
            ( self.robots.get("geode", 0) * (n+1) ) + # existing robots times rounds remaining
            geodes_per_robot # building a new geode robot for each remaining round
        )

    def next(self, max_depth):
        # Figure out what we can build, we can always build nothing
        available_builds = set([ None ])
        for robot, costs in self.blueprint.costs.items():
            satisifed = True
            for what, cnt in costs.items():
                if self.inventory.get(what, 0) < cnt:
                    satisifed = False
                    break
            else:
                available_builds.add(robot)

        # Branch for each available build
        for build in reversed((None, "ore", "clay", "obsidian", "geode")):
            if build not in available_builds:
                continue
            new_state = State(self.blueprint, self)
            #new_state.path.append(build)
            
            if build:                    
                for what, cnt in self.blueprint.costs[build].items():
                    new_state.inventory[what] = new_state.inventory.get(what, 0) - cnt
                    assert new_state.inventory.get(what, 0) >= 0

            for robot, count in new_state.robots.items():
                new_state.inventory[robot] = new_state.inventory.get(robot, 0) + count

            # Finish Building
            if build:
                new_state.robots[build] = new_state.robots.get(build, 0) + 1

            yield build, new_state

def analyze(blueprint, max_depth):
    max_ore = max([ xx.get("ore", 0) for xx in blueprint.costs.values() ])
    max_clay = max([ xx.get("clay", 0) for xx in blueprint.costs.values() ])
    max_obsidian = max([ xx.get("obsidian", 0) for xx in blueprint.costs.values() ])
    
    final_states = []
    max_score = 0
    first_geode = None
    max_geodes = 0
    cur_depth = 0
    prune_cnt = 0
    max_state = None
    earliest_geode = None
    root = State(blueprint)
    root._explored = True

    mode = "dfs"
    if mode in ("dfs", "bfs"):
        Q = collections.deque()
        Q.append(root)
    else:
        Q = []
        heapq.heappush(Q, (0, root))

    while len(Q) > 0:
        if mode == "bfs":
            v = Q.popleft()  # BFS
        elif mode == "dfs":
            v = Q.pop() # DFS
        elif mode == "pdfs":
            v = heapq.heappop(Q)[-1]
        else:
            raise ValueError

        if mode == "bfs" and v.depth > cur_depth:
            print(f"At depth {v.depth} pruned {prune_cnt}  {len(Q)}")
            cur_depth = v.depth
            prune_cnt = 0
        
        if v.score > max_score:
            print("New score max", v.score, v.depth, prune_cnt)
            max_score = max(v.score, max_score, prune_cnt)
        if v.inventory.get("geode", 0) > max_geodes:
            print("New geodes max", v.inventory.get("geode", 0), max_geodes)
            max_geodes = max(v.inventory.get("geode", 0), max_geodes, prune_cnt)
            max_state = v

        # Prune
        if max(max_score, max_geodes) >= v.best_score(max_depth):
            prune_cnt += 1
            continue

        #if child.depth > max_ore and sum(child.robots.values()) == 1:
        #    prune_cnt += 1
        #    continue

        #if child.depth > 20 and child.robots.get("geode", 0) == 0:
        #    prune_cnt += 1
        #    continue

        if v.depth >= max_depth:
            continue

        for build, child in v.next(max_depth):
            # Child is the state *after* the minute elapses.
            # Score is the number of geodes plus the number of geodes that would be produced
            # in the future
            n = max_depth - child.depth
            child.score = child.inventory.get("geode", 0) + child.robots.get("geode", 0) * n

            # Search
            if mode in ("bfs", "dfs"):
                Q.append(child)
            else:
                heapq.heappush(Q, (-1*child.score, child))


    return max_state, max_geodes

def part2(puzzle_input):
    blueprints = read_input(puzzle_input)
    total_quality = [1,1,1]
    for ii, blueprint in enumerate(blueprints[0:1]):
        print("-"*40)
        max_state, max_geodes = analyze(blueprint, 32)

        if max_state == None:
            print("RESULTS", ii+1, 0, 0)
            total_quality[ii] = 0
        else:
            print("RESULTS", ii+1, max_geodes, (ii+1)*max_geodes)
            print(max_state.inventory)
            print(max_state.robots)
            print(max_state.path)
            total_quality[ii] = max_geodes
            
    print(total_quality)
    return total_quality[0] * total_quality[1] * total_quality[2]

def test_part2():
    ans = part2(os.path.join(os.path.join("..", "dat", "day19_test.txt")))
    print(ans)
    assert ans == (56*62)

test_part2()

#print("X"*40)
#ans = part2(os.path.join(os.path.join("..", "dat", "day19.txt")))
#print(ans) # NOT 5130 (too low)

----------------------------------------
New score max 1 31 3335705
RESULTS 1 0 0
[0, 1, 1]
0


AssertionError: 

In [3]:


class State():
    def __init__(self, prior_state=None):
        self.depth = 0
        self.inventory = {}
        self.robots = { "ore": 1 }
        self.path = []
        self.prior_state = prior_state
        self.discovered = False
        if prior_state is not None:
            self.depth = prior_state.depth + 1
            self.inventory.update(prior_state.inventory)
            self.robots.update(prior_state.robots)
            self.path.extend(prior_state.path)

def part1(puzzle_input):
    blueprints = read_input(puzzle_input)
    total_quality = 0
    for ii, blueprint in enumerate(blueprints):
        states = collections.deque()
        states.append(State())

        max_ore = max([ xx.get("ore", 0) for xx in blueprint.costs.values() ])
        max_clay = max([ xx.get("clay", 0) for xx in blueprint.costs.values() ])
        max_obsidian = max([ xx.get("obsidian", 0) for xx in blueprint.costs.values() ])
        

        final_states = []
        most_geodes = [ 0 ] * 24
        while len(states) > 0:
            state = states.popleft()
            state.discovered = True
            #print("Checking depth %s" % state.depth)
            if state.depth == 24:
                final_states.append(new_state)
                continue

            if state.inventory.get("geode", 0) < most_geodes[state.depth]:
                continue
            else:
                most_geodes[state.depth] = max(most_geodes[state.depth], state.inventory.get("geode", 0))
            
            #if state.inventory.get("ore", 0) > max_ore and state.depth < (24 - max_ore):
                # prune
            #    continue
            #if state.inventory.get("clay", 0) > max_clay and state.inventory.get("ore", 0) > blueprint.costs['obsidian'].get('ore', 0) and state.depth < (24 - max_clay):
                # prune
            #    continue
            #if state.inventory.get("obsidian", 0) > max_obsidian and state.inventory.get("ore", 0) > blueprint.costs['geode'].get('ore', 0) and state.depth < (24 -max_obsidian):
                # prune
            #    continue

            # Figure out what we can build, we can always build nothing
            available_builds = set([ None ])
            for robot, costs in blueprint.costs.items():
                satisifed = True
                for what, cnt in costs.items():
                    if state.inventory.get(what, 0) < cnt:
                        satisifed = False
                        break
                else:
                    available_builds.add(robot)

            # Branch for each available build
            for build in ("geode", "obsidian", "clay", "ore", None):
                if build not in available_builds:
                    continue

                new_state = State(state)
                new_state.path.append(build)
                if build:                    
                    for what, cnt in blueprint.costs[build].items():
                        new_state.inventory[what] = new_state.inventory.get(what, 0) - cnt
                        assert new_state.inventory.get(what, 0) >= 0

                for robot, count in new_state.robots.items():
                    new_state.inventory[robot] = new_state.inventory.get(robot, 0) + count

                # Finish Building
                if build:
                    new_state.robots[build] = new_state.robots.get(build, 0) + 1


                states.appendleft(new_state)

                            
        max_geodes = 0
        max_state = None
        for state in final_states:
            #print(state.inventory, state.path)
            if state.inventory.get("geode", 0) > max_geodes:
                max_state = state
                max_geodes = state.inventory.get("geode", 0)

        print("-"*40)
        if max_state == None:
            print("RESULTS", ii+1, 0, 0)
        else:
            print("RESULTS", ii+1, max_geodes, (ii+1)*max_geodes)
            print(max_state.inventory)
            print(max_state.robots)
            print(max_state.path)
            total_quality += ((ii+1)*max_geodes)

    return total_quality

def test_part1():
    ans = part1(os.path.join(os.path.join("..", "dat", "day19_test.txt")))
    print(ans)
    assert ans == 33

test_part1()

#print("X"*40)
#ans = part1(os.path.join(os.path.join("..", "dat", "day19.txt")))
#print(ans) # NOT 1174, NOT 1394 (1703!!!)

KeyboardInterrupt: 